In [1]:
import pandas as pd
import numpy as np
import gc

## Загрузка и объединение данных

In [3]:
df_app = pd.read_csv('./data/application_train.csv')
df_bureau = pd.read_csv('./data/df_bureau_raw.csv')
df_previous = pd.read_csv('./data/df_previous_raw.csv')

print(f"Размеры датасетов:")
print(f"application_train: {df_app.shape}")
print(f"df_bureau_raw: {df_bureau.shape}")
print(f"df_previous_raw: {df_previous.shape}")

Размеры датасетов:
application_train: (307511, 122)
df_bureau_raw: (307511, 334)
df_previous_raw: (307511, 1125)


### Проверка пересечений колонок

In [ ]:
def get_column_overlap(df1, df2, df3, id_column='SK_ID_CURR'):
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    cols3 = set(df3.columns)
    
    overlap = {
        'app_bureau': cols1.intersection(cols2) - {id_column},
        'app_previous': cols1.intersection(cols3) - {id_column},
        'bureau_previous': cols2.intersection(cols3) - {id_column}
    }
    return overlap

overlap = get_column_overlap(df_app, df_bureau, df_previous)
print("Пересечения колонок (без SK_ID_CURR):")
print(f"app ∩ bureau: {len(overlap['app_bureau'])} колонок")
print(f"app ∩ previous: {len(overlap['app_previous'])} колонок")
print(f"bureau ∩ previous: {len(overlap['bureau_previous'])} колонок")

Пересечения колонок (без SK_ID_CURR):
app ∩ bureau: 121 колонок
app ∩ previous: 121 колонок
bureau ∩ previous: 121 колонок


### Объединение с сохранением оригинальных колонок из df_app

In [ ]:
def smart_merge(main_df, add_df, suffix, id_column='SK_ID_CURR'):
    main_cols = set(main_df.columns)
    add_cols = set(add_df.columns)
    
    # Колонки для удаления (уже есть в main_df, кроме ID)
    cols_to_drop = add_cols.intersection(main_cols) - {id_column}
    
    # Удаляем дубликаты из add_df
    add_df_clean = add_df.drop(columns=list(cols_to_drop))
    
    # Объединяем
    merged_df = pd.merge(main_df, add_df_clean, on=id_column, how='left')
    return merged_df

# Объединяем последовательно
df_merged = smart_merge(df_app, df_bureau, '_bureau')
df_merged = smart_merge(df_merged, df_previous, '_previous')

print(f"\nИтоговый размер: {df_merged.shape}")
print(f"Колонок: {len(df_merged.columns)}")

# Очистка памяти
del df_app, df_bureau, df_previous
gc.collect()


Итоговый размер: (307511, 1337)
Колонок: 1337


1036

## Оптимизация типов данных

In [ ]:
def optimize_dtypes(df):
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.select_dtypes(include=['int', 'float']).columns:
        col_type = df[col].dtype
        
        if 'int' in str(col_type):
            c_min, c_max = df[col].min(), df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        elif 'float' in str(col_type):
            c_min, c_max = df[col].min(), df[col].max()
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Экономия памяти: {start_mem:.2f}MB → {end_mem:.2f}MB ({(start_mem-end_mem)/start_mem*100:.1f}%)")
    return df

df_optimized = optimize_dtypes(df_merged.copy())

C:\Users\User\AppData\Local\Temp\ipykernel_10896\4290237338.py:18: RuntimeWarning: overflow encountered in cast
  if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
C:\Users\User\AppData\Local\Temp\ipykernel_10896\4290237338.py:18: RuntimeWarning: overflow encountered in cast
  if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
C:\Users\User\AppData\Local\Temp\ipykernel_10896\4290237338.py:18: RuntimeWarning: overflow encountered in cast
  if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
C:\Users\User\AppData\Local\Temp\ipykernel_10896\4290237338.py:18: RuntimeWarning: overflow encountered in cast
  if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
C:\Users\User\AppData\Local\Temp\ipykernel_10896\4290237338.py:18: RuntimeWarning: overflow encountered in cast
  if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
C:\Users\User\AppData\Local\Temp\ipykernel_10896\4290237338.

Экономия памяти: 3136.77MB → 1006.78MB (67.9%)


Сохраняем промежуточный результат

In [ ]:
df_optimized.to_csv('df_merged_optimized.csv', index=False)
print("Сохранено: df_merged_optimized.csv")

df = pd.read_csv('df_merged_optimized.csv')
del df_merged, df_optimized
gc.collect()

Сохранено: df_merged_optimized.csv


0

## Анализ структры данных

In [ ]:
print("БАЗОВЫЙ АНАЛИЗ:")
print(f"Размер: {df.shape[0]} строк, {df.shape[1]} колонок")
print(f"ID колонка: {'SK_ID_CURR' in df.columns}")
print(f"Target колонка: {'TARGET' in df.columns}")

if 'TARGET' in df.columns:
    target_dist = df['TARGET'].value_counts(normalize=True)
    print(f"\nРаспределение TARGET:")
    print(f"0: {target_dist[0]:.2%}")
    print(f"1: {target_dist[1]:.2%}")

# Типы данных
print(f"\nТипы данных:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"{dtype}: {count}")

# Категориальные колонки
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nКатегориальных колонок: {len(cat_cols)}")
if cat_cols:
    print("Примеры:", cat_cols[:5])

БАЗОВЫЙ АНАЛИЗ:
Размер: 307511 строк, 1337 колонок
ID колонка: True
Target колонка: True

Распределение TARGET:
0: 91.93%
1: 8.07%

Типы данных:
float64: 1279
int64: 42
object: 16

Категориальных колонок: 16
Примеры: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE']


### Анализ пропусков

In [ ]:
missing_stats = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
high_missing = missing_stats[missing_stats > 50]

print(f"Колонок с пропусками: {(missing_stats > 0).sum()}")
print(f"Колонок с >50% пропусков: {len(high_missing)}")

if len(high_missing) > 0:
    print("\nТоп-5 колонок с наибольшими пропусками:")
    for col, perc in high_missing.head().items():
        print(f"{col}: {perc:.1f}%")

Колонок с пропусками: 1281
Колонок с >50% пропусков: 483

Топ-5 колонок с наибольшими пропусками:
client_credit_AMT_PAYMENT_CURRENT_mean_max: 80.1%
client_credit_AMT_PAYMENT_CURRENT_min_max: 80.1%
client_credit_AMT_PAYMENT_CURRENT_min_mean: 80.1%
client_credit_AMT_PAYMENT_CURRENT_mean_min: 80.1%
client_credit_AMT_PAYMENT_CURRENT_mean_mean: 80.1%


## Разделение на train/val/test

In [10]:
from sklearn.model_selection import train_test_split

# Отделяем ID и target
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])
y = df['TARGET']
ids = df['SK_ID_CURR']

# Разделение: 60% train, 20% val, 20% test
X_temp, X_test, y_temp, y_test, ids_temp, ids_test = train_test_split(
    X, y, ids, test_size=0.2, stratify=y, random_state=42
)

X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(
    X_temp, y_temp, ids_temp, test_size=0.25, stratify=y_temp, random_state=42
)  # 0.25 * 0.8 = 0.2 от общего

print("Разделение данных:")
print(f"Train: {X_train.shape[0]} ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Val: {X_val.shape[0]} ({X_val.shape[0]/len(df)*100:.1f}%)")
print(f"Test: {X_test.shape[0]} ({X_test.shape[0]/len(df)*100:.1f}%)")

Разделение данных:
Train: 184506 (60.0%)
Val: 61502 (20.0%)
Test: 61503 (20.0%)


### Сохраняем разделенные данные

In [ ]:
train_data = pd.concat([ids_train, y_train, X_train], axis=1)
val_data = pd.concat([ids_val, y_val, X_val], axis=1)
test_data = pd.concat([ids_test, y_test, X_test], axis=1)

train_data.to_csv('./data/train_raw.csv', index=False)
val_data.to_csv('./data/val_raw.csv', index=False)
test_data.to_csv('./data/test_raw.csv', index=False)

print("Сохранено:")
print("- train_raw.csv")
print("- val_raw.csv")
print("- test_raw.csv")

Сохранено:
- train_raw.csv
- val_raw.csv
- test_raw.csv


In [12]:
del df, X, y, X_temp, X_test, y_temp, y_test, ids_temp, ids_test, X_train, X_val, y_train, y_val, ids_train, ids_val, train_data, val_data, test_data
gc.collect()

28

# Вывод

Аггрегирвоали все данные из всех файлов в одни датасет. Оптимизирвоали типы данных для призанков, чтобы уменьшить занимаемое место в памяти. Разделили данные на train/valid/test применим стратификацию. Сохранили файл со всеми данными, а также отдельные файлы для train/valid/test, чтобы было удобнее загружать и обрабатывать данные.